# 大语言模型原理简介

**大语言模型**（**large language model**，**LLM**）的出现使得**人工智能**（**artificial intelligence**，**AI**）的发展进入了新的纪元，特别是OpenAI的GPT系列模型经过不断迭代进化，让人们看到了基于大语言模型的人工智能的无限可能。

实际上，大语言模型的出现并非突然，而是有深厚的“小语言”模型的基础。实际上正如**GPT**（**Generative Pre-trained Transformers**）包括**BERT**（**Bidirectional Encoder Representations from Transformers**）名称中所暗示的，两者都是基于谷歌的几位科学家2017年的论文《Attention is All You Need》所提出的Transformer架构：

![Transformer架构](pic/transformer.png)

而这一架构的提出则是将之前的**嵌入**（**embedding**）这一方法（如Word2Vec）进行了拓展。

在传统的嵌入方法中，句子首先被分解为一些**标记**（**token**）：
* 对于英文而言，标记可以是一个单词，或者是比单词更细的成分（比如把shocked分开为shock和ed）；
* 对于中文而言，通常每个字是一个token，或者可以通过分词技术进行分词（但在LLM中似乎没有必要）。
然后每个标记被转化为一个向量，从而把文本转化为了一个可以被计算的对象。

然而以上方法有其弊端：有的词天然是多义的，无法考虑上下文。比如，“学长可以写出更长的文章”这个句子中，两个“长”甚至连读音都不相同，但是在之前的嵌入模型中，“长”字可能被翻译为同一个向量。当然也许可以使用分词和n-gram的方法部分解决这一问题，然而会导致模型的复杂度急剧上升。

为此，Transformer架构中创造性地使用了**注意力**（**attention**）机制解决这一问题。我们在这里简要介绍一下注意力机制的简单直觉，不再对其数学部分做更多的解读，理解这一算法背后的直觉可以帮助我们理解大语言模型以及更好地使用大语言模型。

## 注意力机制

正如字面意思，注意力机制是模仿了人脑的注意力。比如，当我们阅读一段文字：

> 在我的后园，可以看见墙外有两株树，一株是枣树，还有一株也是枣树。

当我们看到“两棵树”，自然会在后文中**查找**（**Query**）是哪两棵，我们可能会以“树”这个**关键字**（**Key**）去查找，然后找到了后文的两个“枣树”，进而在脑海中形成了“两颗枣树”这一映像，或者在计算机语言中，也就是“键”所对应的“**值**”（**Value**）。

在Attention中正是使用以上Q、K、V三个工具，通过Q和V两个矩阵相乘，经过Softmax变成一个注意力的概率，再与V相乘得到最终的注意力结果。

由于在一个句子中，根据不同的任务，我们的注意力可能集中在不同的token上面，所以在实际使用时，使用的是“多头注意力”（multi-head attention），即将多个注意力工具合在一起使用。注意力的架构如下图所示：

![Attention](pic/attention.png)

根据不同的任务，注意力可以分为两种：交叉注意力和自注意力。

* 交叉注意力（cross attention）：Q来自于一个序列，而K、V来自于另一个序列：

![Cross-Attention](pic/cross_attention.png)

* 自注意力（self-attention）：Q、K、V来自于同一个序列：

![Self-Attention](pic/self_attention.png)

## LLM的训练过程

注意力机制以及Transformer架构为大语言模型提供了技术保障，然而还有两个对LLM训练至关重要的问题没有解决：

1. 如何训练模型？需要定义好损失函数（训练目标是什么？）
2. 数据源从哪里来？

对第二个问题的回答来源于第一个问题，不同的训练任务、目标需要有不同的损失函数，从而也需要不同的数据。

在Transformer的原文中，主要解决的是翻译问题，那么只需要提供原文和译文，目标函数就是让机器根据原文翻译的译文与人翻译的尽量相近。为此，需要同时使用编码器和解码器。

而在Transformer提出后的一年，有两个工作放弃了编码器-解码器的结构，而是只使用了其中的一种：

* BERT：只使用了编码器
* GPT：只使用了解码器

具体而言，BERT由于只提供编码器作为预训练模型使用，因而只有预训练阶段；而GPT的目的是生成，因而除了预训练，还有有监督微调（supervised fine-tuning）。

在预训练阶段，BERT和GPT使用了类似的策略，其中：

* BERT完成两种任务
  1. 遮蔽语言模型（MLM），类似于完形填空，比如：
    > [CLS]一条大[MASK]波浪宽[SEP]
  2. 下句预测（NSP）：预测下个句子
* GPT主要预测下一个token（NTP）。

预训练之后，GPT还会进行进一步的有监督微调，通过文本分类、问答、自然语言推断等有监督任务继续训练模型。

BERT和GPT都不约而同使用了BooksCorpus的数据集，以及英文维基百科等。后续模型则扩充了使用的数据集。

从最初版本的GPT到GPT3，区别主要在于模型的大小和用于训练的数据量。

## 进化：从GPT到InstructGPT

经过以上的步骤之后，模型基本可以做到“熟读唐诗三百首，不会作诗也会吟”，然而其效果往往差强人意，离人工智能的目标还差的很远，甚至可能会产生对用户有害的输出。为此，OpenAI创造性地开创了InstructGPT。[这个链接](https://openai.com/index/instruction-following/) 解释了InstructGPT的工作流程，核心步骤如下：

![InstructGPT](pic/instructgpt.png)

其核心步骤包括两个：

1. 监督微调（supervised fine-tuning）
2. 通过人类反馈进行强化学习（reinforcement learning from human feedback, RLHF）

从第一个GPT模型只有0.1b参数（b代表billion，十亿）开始，到2020年GPT-3的175b参数，再到2022年InstructGPT推出，从而进化到GPT-3.5（同样为175b参数），大模型的逐渐展现出了超乎想象的能力。到现在的GPT-4、GPT-4o，短时间内大模型的推力能力得到了极大的增强。与此同时，各种各样的大模型不断推出，包括Llama、GLM、Bard以及中文的通义千问、文心一言等大模型也有不错的效果。

## 警惕AI幻觉

虽然大语言模型已经表现出了强大的推理能力，然而值得注意的是，这里的推理仍然是在语言层面而不是在逻辑层面。所谓推理，无非是根据上下文不断的对下一个token进行预测，借助于足够大的参数量，大语言模型往往可以“背过”很多答案，但是要求大语言模型可以基于逻辑进行推理现在仍然难以实现。当然，从另一个层面说，人类的思考也是借助于语言工具，也许有一天完全基于语言的大模型可以进行逻辑推理，但是现在仍然难以达到。

一个比较简单而又广为人知的例子是问大模型：9.9和9.11哪个大？包括GPT在内的很多模型都回答错误。如果使用一些提问的技巧，GPT甚至会“振振有词”地为错误答案辩护。在一些计算题中同样有这个问题，比如如果问2 * 2等于多少，可以得到正确答案，但是如果问3666 * 5444等于多少，则可能会给出错误答案。

因而，不要完全信任大语言模型给出的结果。在实际使用过程中，应该借助更多的工具帮助大语言模型得到更好的答案，而不是完全依赖这个答案。

## 提示工程

由于大语言模型的不完美，往往我们的问题得不到最满意的回答，然而通过提高提示词的质量往往可以得到更好的回答。为此，提供正确的、高质量的提示词就可以让我们对AI工具的使用事半功倍，这被称为提示工程（prompt engineering）。常见的方法有：

* 提供详细、明确的上下文
  > 有一家公司为大飞机生产起落架，是不是新质生产力？
  > 新质生产力是创新起主导作用，摆脱传统经济增长方式、生产力发展路径，具有高科技、高效能、高质量特征，符合新发展理念的先进生产力质态。它由技术革命性突破、生产要素创新性配置、产业深度转型升级而催生。以劳动者、劳动资料、劳动对象及其优化组合的跃升为基本内涵，以全要素生产率大幅提升为核心标志，特点是创新，关键在质优，本质是先进生产力。根据以上定义，请问为大飞机生产起落架的公司符不符合新质生产力的要求？
* 明确任务
  > 请为比亚迪公司是否属于新质生产力打分，1-10分。
* 明确角色
  > 如果你是一个计量经济学老师，请写一个教学计划。
* 零样本思维链策略（zero-shot-CoT strategy）：让LLM逐步思考、给出过程和步骤
  > 3666 * 5444等于多少？请给出计算步骤
* 少样本学习（few-shot learning）甚至单样本学习（one-shot learning）
  > 1025=1，20480=2，360560101=3，5023512004=？
* 格式化输出
  > 请为比亚迪公司是否属于新质生产力打分，1-10分，必须为数字
* 重复指示：提示词中多次加入相同的指令，但是表述方式不相同，也可以通过负面来提示
  > 你作为投资人，需要考察比亚迪公司的新质生产力发展情况，需要进行一个打分。请你根据你所知道的情况，对比亚迪公司进行分析，针对不好的地方进行扣分，1-10分。不要给出理由，必须给出数字。
* 思维链、思维树提示：将复杂问题分解

# Ollama的部署与使用

通常人们喜欢使用网页版大模型，然而很多时候这些大模型的使用缺乏定制，同时在任务量比较大时难以批量处理。为此，一个好的办法是直接使用大模型的接口。

比如，OpenAI提供了使用他们大模型的接口，在Python中可以直接使用
```bash
python3 -m pip install openai
```
进行安装和使用。当然，在使用前需要完成注册并获得秘钥。具体使用方法可以参考《大模型应用开发极简入门》一书。

OpenAI的工具有其优点，比如效果好，甚至可以定制化，但是也有其缺点，除了注册麻烦之外，使用OpenAI的成本也是非常高的。因而，在一些场景下，使用开源的大模型也许是一个不错的选择。

实际上开源大模型已经有很多，也达到了不错的效果。常见的有：

* Llama：Meta公司的大语言模型[Github](https://github.com/meta-llama/llama3) [Huggingface](https://huggingface.co/meta-llama)
* gemma2：Google的大语言模型[Huggingface](https://huggingface.co/google)
* Phi-3: 微软的大语言模型[Github](https://github.com/microsoft/Phi-3CookBook) [Huggingface](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf)
* 通义千问：阿里巴巴的大语言模型[Github](https://github.com/QwenLM) [Huggingface](https://huggingface.co/Qwen)
* GLM：智谱AI的大语言模型[Github](https://github.com/THUDM/GLM-4) [Huggingface](https://huggingface.co/THUDM)
* ......

此外还有一些多模态的模型（如llava）以及特定领域的模型（如代码生成和数学领域等）。

根据实际使用，开源大模型的完成度目前已经非常高，在特定领域甚至有一些优势，因而如果受限于条件和成本，特别是如果有微调大模型的需求，开源大模型也许是一个不错的选择。

## Ollama的部署

![Ollama](pic/ollama.png)

直接部署开源大模型往往比较麻烦，如果只是使用大模型而不是对大模型进行更深层次开发的话，使用一些成熟的部署工具就非常方便了。

这里以Ollama为例，介绍如何在本地部署大模型。

### 硬件环境

神经网络特别是大模型的训练和推理非常依赖显卡（GPU），而对于大模型而言，显存是重要的约束。比如，使用Ollama运行一个通义千问的14b模型，需要占用约11G显存，而往往很多人电脑的内存还到不了这个水平。

Ollama的一个好处是可以不依赖GPU就运行大模型，当然同样也需要足够大的内存。但是拥有一块具有足够大显存的显卡仍然是第一选择，因为GPU不管是训练还是推理，速度都远远快于CPU。

### 软件环境

* 操作系统：不限操作系统，但是推荐使用Linux
* Docker：一种称为“容器”的技术，类似于虚拟机但是有很大不同：使用Linux的命名空间（Namespaces）和控制组（Cgroups）完成容器和系统、容器之间的隔离，且不像虚拟机那样有大量性能消耗。Docker技术的使用极大方便了复杂环境的部署。
  * Windows和Mac也可以使用Docker，但是依靠的是虚拟一个Linux
  * Windows下也可以考虑通过WSL（Windows Subsystem for Linux）使用，但是WSL2同样基于虚拟机

### 安装

Ollama的[GitHub](https://github.com/ollama/ollama)上提供了下载链接：
* [Windows](https://ollama.com/download/OllamaSetup.exe)
* [Mac](https://ollama.com/download/Ollama-darwin.zip)
* Linux:
  ```bash
  curl -fsSL https://ollama.com/install.sh | sh
  ```

此外，Ollama还可以通过Docker安装：[镜像地址](https://hub.docker.com/r/ollama/ollama)，注意如果使用Docker安装需要区分CPU版和GPU版，GPU版需要安装NVIDIA的Container Toolkit。

### 拉取、运行模型

Ollama安装好之后，可以通过pull来拉取特定的模型，Ollama的官网提供了[模型列表](https://ollama.com/library)，比如拉取llama3.1：
```bash
sudo ollama pull llama3.1:8b
```
其中llama3.1是模型的名称，冒号后面的8b代表的是模型的大小。或者通义千问：
```bash
sudo ollama pull qwen2.5:14b
```

拉取好之后可以通过ollama run命令直接运行（不拉取直接run也会自动完成拉取步骤）：
```bash
ollama run qwen2.5:14b
```
然后就可以对话了。结束对话可以使用ctrl-D。

### 服务

对于Linux，安装好Ollama后会启动一个Ollama服务，该服务服务于**11434端口**，默认只侦听来自于本机(127.1.1.1或localhost)的访问。

如果需要设置Ollama能够接受所有的访问，需要对配置文件进行修改，比如在Linux中，需要
```bash
sudo vim /etc/systemd/system/ollama.service
```
在[Service]中加入：
```bash
Environment="OLLAMA_HOST=0.0.0.0"
```
然后
```bash
sudo systemctl daemon-reload
sudo systemctl restart ollama.service
```
即可。

Ollama的API使用方法可以在其[Github主页链接](https://github.com/ollama/ollama/blob/main/docs/api.md)上找到，我们接下来也会介绍主要接口的使用方法。

## 嵌入

大语言模型的一个基础就是嵌入。如前所述，借助于注意力机制，大语言模型可以通过上下文改变初始的token嵌入向量。传统上，BERT作为编码器是比较适合做嵌入工作的，不过基于解码器结构的生成式大语言模型也可以输出嵌入向量。

嵌入向量通常有如下作用：

* 在**检索增强生成**（**Retrieval-augmented Generation**）中作为检索依据
* 作为预训练模型继续参与接下来的训练（文本分类等问题）
* 在经济学中，结合双重机器学习，将文本作为控制变量时使用嵌入向量（如Dube，2020使用了Doc2Vec的嵌入结果）

借助于Ollama的接口，我们可以在不接触大模型源码的前提下直接得到嵌入向量的结果。

In [1]:
import urllib3
import json
import numpy as np


def get_ollama_embedding(text,
                         model='qwen2.5:14b',
                         url='127.0.0.1',
                         port='11434'):
    """
    获取ollama的embedding
    :param text: 输入文本
    :return: embedding
    """
    url = f'http://{url}:{port}/api/embed'
    data = json.dumps({"model": model, "input": text})
    # 使用POST方法发送请求
    http = urllib3.PoolManager()
    response = http.request('POST',
                            url,
                            headers={'Content-Type': 'application/json'},
                            body=data)
    res = json.loads(response.data.decode('utf-8'))
    embedding = np.array(res['embeddings'])
    if embedding.ndim == 2:
        embedding = embedding.squeeze()
    return embedding, res


embedding, res = get_ollama_embedding(
    "大江东去，浪淘尽，千古风流人物。故垒西边，人道是，三国周郎赤壁。乱石穿空，惊涛拍岸，卷起千堆雪。江山如画，一时多少豪杰。遥想公瑾当年，小乔初嫁了，雄姿英发。羽扇纶巾，谈笑间，樯橹灰飞烟灭。故国神游，多情应笑我，早生华发。人生如梦，一尊还酹江月。"
)
print(res)
print(embedding.shape)
print(embedding)

{'model': 'qwen2.5:14b', 'embeddings': [[-0.004463677, 0.0006587918, 0.00081877154, 0.0040618596, 0.0030021095, 0.0034227576, -0.0050876336, -0.0012199628, -0.005698439, 0.0033160122, -0.002683941, 0.0073378948, 0.005158564, 0.003652678, -0.0025515675, 0.0052614613, -0.0018691926, 0.007824965, -0.006993309, 0.0038674672, -0.0045476174, 0.0055523473, 0.0052257976, 0.0005257912, 0.009152101, -0.0014755365, -0.0014519205, 0.016857535, -0.0064902427, -0.0060018008, -0.0013289016, 0.0009453353, -0.0076018614, -0.01241445, -0.0003883615, -0.0063787135, 0.0017416257, -0.0037255443, -0.012341534, -0.01361946, 0.0023451424, 0.013120032, 0.0067442325, -0.01031063, -0.011762094, -0.0004029979, -0.004612, 0.0034536219, 0.0023227914, 0.0077848868, 0.005588273, -0.007718209, -0.0014235773, 0.007116389, -0.006965277, -0.0034684776, -0.007112187, -0.0013103319, -0.0009523148, 0.0037913609, 0.004969782, -0.0034918867, 0.0069435467, -0.0044993074, 0.0069893585, -0.0048034983, 0.0042289575, -0.008320726,

作为一个简单的应用，我们比较一下苏轼的两首词分别与李清照、毛主席的词的余弦相似度：

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


embedding1, _ = get_ollama_embedding(
    "明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间。转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。"
)
embedding2, _ = get_ollama_embedding(
    "寻寻觅觅，冷冷清清，凄凄惨惨戚戚。乍暖还寒时候，最难将息。三杯两盏淡酒，怎敌他、晚来风急！雁过也，正伤心，却是旧时相识。满地黄花堆积，憔悴损，如今有谁堪摘？守着窗儿，独自怎生得黑！梧桐更兼细雨，到黄昏、点点滴滴。这次第，怎一个愁字了得！"
)
embedding3, _ = get_ollama_embedding(
    "北国风光，千里冰封，万里雪飘。望长城内外，惟余莽莽；大河上下，顿失滔滔。山舞银蛇，原驰蜡象，欲与天公试比高。须晴日，看红装素裹，分外妖娆。江山如此多娇，引无数英雄竞折腰。惜秦皇汉武，略输文采；唐宗宋祖，稍逊风骚。一代天骄，成吉思汗，只识弯弓射大雕。俱往矣，数风流人物，还看今朝。"
)

print("苏东坡v.s.苏东坡",cosine_similarity(embedding, embedding1))
print("苏东坡v.s.李清照",cosine_similarity(embedding, embedding2))
print("苏东坡v.s.毛主席",cosine_similarity(embedding, embedding3))
print("苏东坡v.s.李清照",cosine_similarity(embedding1, embedding2))
print("苏东坡v.s.毛主席",cosine_similarity(embedding1, embedding3))
print("李清照v.s.毛主席",cosine_similarity(embedding2, embedding3))

苏东坡v.s.苏东坡 0.979588190472293
苏东坡v.s.李清照 0.9350070203571802
苏东坡v.s.毛主席 0.9600337289203775
苏东坡v.s.李清照 0.9359491468475117
苏东坡v.s.毛主席 0.9582751064187163
李清照v.s.毛主席 0.9391653647395104


可以看到，我们可以几乎无样本地得到一个很好的文本分析结果。

## 对话

嵌入只是小试牛刀，真正的重头戏是对话。

## 微调大模型

# Dify的部署和使用